<a href="https://colab.research.google.com/github/KonstantinBurkin/Sales_forecasting/blob/main/sales_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>
<center><h1>Sales Forecasting</h1></center>
<br>

<center><h3>Konstantin Burkin</h3></center>

<center> <i>22 April 2022 </i> </center>

<br>  
<h2>Table of contents</h2> 

---
- <a href=#Introduction>Introduction</a> <br>
  - <a href=#Assignment>Assignment</a> <br>
  - <a href=#Brief-outline>Brief outline</a> <br>
- <a href=#Import-of-data-and-Python-libraries>Import of data and Python libraries</a> <br>
- <a href=#Exploratory-Data-Analysis>Exploratory Data Analysis</a> <br>
  - <a href=#Statistical-data-description>Statistical data description</a> <br>
  - <a href=#Data-visualisation>Data visualisation</a> <br>
- <a href=#Prediction>Prediction</a> <br>
- <a href=#Building-ML-models>Building ML models</a> <br>
- <a href=#Results>Results</a> <br>

<br> 
<h2 id="Introduction">Introduction</h2>  

--- 

<div align="justify">This work was created and submitted as a final course project for MSU course "Machine learning for applied problems" in spring semester 2022. The project was written in <a href="https://https://colab.research.google.com">Google Colab</a>, using Python <i>version 3.7.13</i>. This work is available in <a href="https://github.com/KonstantinBurkin/Machine_Learning_Project">my Github repository</a>, where it is possible to download Colab Notebook, check the code and reproduce my work.</div> <br>

<h3 id="Assignment">Assignment</h3> 

<div align="justify">
The goal is to create predictions to evaluate the number of sales in Delivery Club for a week in February. This week was described by "test" dataframe, that contained information about the date, weather conditions, product types, stores, and their locations. The history of the sales for previous 7 months is available in the "train" dataframe. The predictions should be made with any kind of ML model. However, predictions should not exceed 4.1 points of mean absolute error metric. Both dataframes describing Delivery Club sales from 160 stores in 10 russian cities were provided by the supervisor.</div> <br>

<div align="justify">The result of this project is a dataframe containing id of products and corresponding number of predicted sales. The accuracy of predictions was evaluated using <a href="https://contest.yandex.ru/">Yandex.contest</a> platform.

<br>  

<h3 id="Brief-outline">Brief outline</h3> 

<ul>
  <li>Setup notebook environment</li><ul>
  <li>Import data and libraries</li>
  <li>Dataframe conversion to English format</li></ul>
  <li>Exploration of data</li>
  <ul>
  <li>Data types, number of unique values, description of columns, missing values analysis</li>
  <li>Statistical evaluation of numeric columns, initial correlation between columns</li></ul>
  <li>Visualisation of patterns in the dataframe</li>
  <!-- <li>between sales column and other columns.</li> -->
  <li>Data transformation. Visualisation of dependencies between sales column and other columns</li>
  <li>Comparison of ML models to choose the one with minimal absolute error</li>
  <li>Building predictions and visualisation the result</li>
</ul> 

</div> 

<br> 
<h2 id="Import-of-data-and-Python-libraries">Import of data and Python libraries</h2>  

--- 

<div align="justify">The following data analysis includes several Python libraries for data analysis, builduing ML models, ploting data, etc: </div>  

- Numpy
- Pandas
- Plotly
- Sklearn
- Seaborn  
- Google colab   

<div align="justify">Two dataframes were downloaded and read (train.csv and test.csv) from my Github repository. Both dataframes are available on my Github <a href="https://github.com/KonstantinBurkin/Sales_forecasting/tree/main/data">page</a>. The train dataframe is used to examine data, find patterns and factors that correlate with the number of sales, add new variables and test ML models. The test dataframe is used to build predictions that will be evaluated later in Yandex.contest.</div>

In [ ]:
# Import libraries

import numpy as np
import pandas as pd
import sklearn
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from google.colab import output
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from pandas import DatetimeIndex as dt
from sklearn.preprocessing import StandardScaler
from google.colab import files
import IPython
from IPython.display import HTML, display

# ignore warnings when graphs are plotted
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Download data from Github repo

!wget --no-cache --backups=1 {"https://raw.githubusercontent.com/KonstantinBurkin/Machine_Learning_Project/main/data/train.csv"}
!wget --no-cache --backups=1 {"https://raw.githubusercontent.com/KonstantinBurkin/Machine_Learning_Project/main/data/test.csv" }
output.clear()

In [ ]:
# Read csv files
train = pd.read_csv("train.csv")      # download train data
test = pd.read_csv("test.csv")        # download test data

# Transform city names to english format
train = train.replace(            
    ('Москва', 'Санкт-Петербург', 'Краснодар', 'Самара','Нижний Новгород', 'Ростов-на-Дону', 'Волгоград', 'Воронеж', 'Казань', 'Екатеринбург'),
    ("Moscow", "St.Petersburg", "Krasnodar", "Samara", "Nizhny.Novgorod", "Rostov-on-Don", "Volgograd", "Voronezh", "Kazan", "Yekaterinburg")  )
test = test.replace(            
    ('Москва', 'Санкт-Петербург', 'Краснодар', 'Самара','Нижний Новгород', 'Ростов-на-Дону', 'Волгоград', 'Воронеж', 'Казань', 'Екатеринбург'),
    ("Moscow", "St.Petersburg", "Krasnodar", "Samara", "Nizhny.Novgorod", "Rostov-on-Don", "Volgograd", "Voronezh", "Kazan", "Yekaterinburg")  )

# Transform weather description to english format
train = train.replace(            
    ('переменная облачность, небольшой дождь', 'переменная облачность', 'облачно, небольшой дождь', 'дождь, гроза', 'облачно, без существенных осадков',
     'переменная облачность, дождь', 'дождь', 'облачно', 'ясно', 'облачно, небольшой снег', 'переменная облачность, небольшие осадки', 'облачно, небольшие осадки', 
     'снег', 'метель', 'осадки', 'переменная облачность, небольшой снег'),
    ('partly cloudy, light rain', 'partly cloudy', 'cloudy, light rain', 'rain, thunderstorm', 'cloudy, no significant precipitation', 
     'partly cloudy, rain', 'rain', 'cloudy', 'clear', 'cloudy, light snow', 'partly cloudy, light precipitation', 'cloudy, slight precipitation', 
     'snow', 'blizzard', 'rainfall', 'partly cloudy, light snow')  )
test = test.replace(            
    ('переменная облачность, небольшой дождь', 'переменная облачность', 'облачно, небольшой дождь', 'дождь, гроза', 'облачно, без существенных осадков',
     'переменная облачность, дождь', 'дождь', 'облачно', 'ясно', 'облачно, небольшой снег', 'переменная облачность, небольшие осадки', 'облачно, небольшие осадки', 
     'снег', 'метель', 'осадки', 'переменная облачность, небольшой снег'),
    ('partly cloudy, light rain', 'partly cloudy', 'cloudy, light rain', 'rain, thunderstorm', 'cloudy, no significant precipitation', 
     'partly cloudy, rain', 'rain', 'cloudy', 'clear', 'cloudy, light snow', 'partly cloudy, light precipitation', 'cloudy, slight precipitation', 
     'snow', 'blizzard', 'rainfall', 'partly cloudy, light snow')  )

In [ ]:
# # the code is represented in the table below
# train.head()

<center><p>The head of the dataframe</p> 

<style type="text/css">
.tg  {border:none;border-collapse:collapse;border-color:#ccc;border-spacing:0;}
.tg td{background-color:#fff;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#f0f0f0;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{border-color:inherit;font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-abip{background-color:#f9f9f9;border-color:inherit;text-align:center;vertical-align:top}
.tg-sort-header::-moz-selection{background:0 0}
.tg-sort-header::selection{background:0 0}.tg-sort-header{cursor:pointer}
.tg-sort-header:after{content:'';float:right;margin-top:7px;border-width:0 5px 5px;border-style:solid;
  border-color:#404040 transparent;visibility:hidden}
.tg-sort-header:hover:after{visibility:visible}
.tg-sort-asc:after,.tg-sort-asc:hover:after,.tg-sort-desc:after{visibility:visible;opacity:.4}
.tg-sort-desc:after{border-bottom:none;border-width:5px 5px 0}@media screen and (max-width: 767px) {.tg {width: auto !important;}.tg col {width: auto !important;}.tg-wrap {overflow-x: auto;-webkit-overflow-scrolling: touch;}}</style>
<div class="tg-wrap"><table id="tg-Hg418" class="tg">
<thead>
  <tr>
    <th class="tg-7btt"><span style="font-weight:bold">id</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">date</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">city_name</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">store_id</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">category_id</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">product_id</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">price</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">weather_desc</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">humidity</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">temperature</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">pressure</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">sales</span></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-abip">1</td>
    <td class="tg-abip">2021-07-29</td>
    <td class="tg-abip">Moscow</td>
    <td class="tg-abip">1</td>
    <td class="tg-abip">1</td>
    <td class="tg-abip">1</td>
    <td class="tg-abip">4.79</td>
    <td class="tg-abip">partly cloudy, light rain</td>
    <td class="tg-abip">61.9375</td>
    <td class="tg-abip">23.1875</td>
    <td class="tg-abip">741.0000</td>
    <td class="tg-abip">26</td>
  </tr>
  <tr>
    <td class="tg-c3ow">2</td>
    <td class="tg-c3ow">2021-07-30</td>
    <td class="tg-c3ow">Moscow</td>
    <td class="tg-c3ow">1</td>
    <td class="tg-c3ow">1</td>
    <td class="tg-c3ow">1</td>
    <td class="tg-c3ow">4.79</td>
    <td class="tg-c3ow">partly cloudy, light rain</td>
    <td class="tg-c3ow">70.2500</td>
    <td class="tg-c3ow">22.1875</td>
    <td class="tg-c3ow">740.3125</td>
    <td class="tg-c3ow">37</td>
  </tr>
  <tr>
    <td class="tg-abip">3</td>
    <td class="tg-abip">2021-07-31</td>
    <td class="tg-abip">Moscow</td>
    <td class="tg-abip">1</td>
    <td class="tg-abip">1</td>
    <td class="tg-abip">1</td>
    <td class="tg-abip">4.79</td>
    <td class="tg-abip">partly cloudy</td>
    <td class="tg-abip">52.6250</td>
    <td class="tg-abip">21.8125</td>
    <td class="tg-abip">741.6250</td>
    <td class="tg-abip">25</td>
  </tr>
  <tr>
    <td class="tg-c3ow">4</td>
    <td class="tg-c3ow">2021-08-01</td>
    <td class="tg-c3ow">Moscow</td>
    <td class="tg-c3ow">1</td>
    <td class="tg-c3ow">1</td>
    <td class="tg-c3ow">1</td>
    <td class="tg-c3ow">4.79</td>
    <td class="tg-c3ow">cloudy, light rain</td>
    <td class="tg-c3ow">87.4375</td>
    <td class="tg-c3ow">20.0625</td>
    <td class="tg-c3ow">743.3125</td>
    <td class="tg-c3ow">26</td>
  </tr>
  <tr>
    <td class="tg-abip">5</td>
    <td class="tg-abip">2021-08-02</td>
    <td class="tg-abip">Moscow</td>
    <td class="tg-abip">1</td>
    <td class="tg-abip">1</td>
    <td class="tg-abip">1</td>
    <td class="tg-abip">4.79</td>
    <td class="tg-abip">partly cloudy</td>
    <td class="tg-abip">66.1875</td>
    <td class="tg-abip">23.4375</td>
    <td class="tg-abip">739.6250</td>
    <td class="tg-abip">22</td>
  </tr>
</tbody>
</table></div></center>

<br> 
<h2 id="Exploratory-Data-Analysis">Exploratory Data Analysis</h2>  

--- 

<h3 id="Statistical-data-description">Statistical data description</h3>  

Two dataframes (train.csv and test.csv) have common list of 11 variables. "Sales" variable is present only in train dataframe.  <br>

In [ ]:
# # the code is represented in the tables below

# # column names and data types
# print(train.info(), "\n")

# # unique values in train dataframe
# print("Number of unique values:")
# print(f"1. id:\t\t\t", train.id.unique().shape[0])
# print(f"2. date:\t\t", train.date.unique().shape[0])
# print(f"3. city_name:\t\t", train.city_name.unique().shape[0])
# print(f"4. store_id:\t\t", train.store_id.unique().shape[0])
# print(f"5. category_id:\t\t", train.category_id.unique().shape[0])
# print(f"6. product_id:\t\t", train.product_id.unique().shape[0])
# print(f"7. price:\t\t", train.	price.unique().shape[0])
# print(f"8. weather_desc:\t", train.weather_desc.unique().shape[0])
# print(f"9. humidity:\t\t", train.humidity.unique().shape[0])
# print(f"10. temperature:\t", train.temperature.unique().shape[0])
# print(f"11. pressure:\t\t", train.pressure.unique().shape[0])
# print(f"12. sales:\t\t", train.sales.unique().shape[0], "\n")

# # shape of the train and test dataframes
# print(f"train shape: {train.shape}\n test shape: {test.shape}\n")

# # list of cities
# print(f"Cities of the dataframe: {train.city_name.unique()}\n")

# # range of dates in the dataframes
# print(f"train.date period: from {train.date.iloc[0]} to {train.date.iloc[-1]}.")
# print(f"test.date period: from {test.date.iloc[0]} to {test.date.iloc[-1]}.\n")

<center><p>Table of variables</p>

<style type="text/css">
.tg  {border:none;border-collapse:collapse;border-color:#ccc;border-spacing:0;}
.tg td{background-color:#fff;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#f0f0f0;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-buh4{background-color:#f9f9f9;text-align:left;vertical-align:top}
.tg .tg-0vih{background-color:#f9f9f9;font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-dzk6{background-color:#f9f9f9;text-align:center;vertical-align:top}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-baqh">№</th>
    <th class="tg-amwm">Variable</th>
    <th class="tg-amwm">Description</th>
    <th class="tg-amwm">Datatype</th>
    <th class="tg-amwm">NAs</th>
    <th class="tg-amwm">Unique values</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0vih"><span style="font-weight:bold">1</span></td>
    <td class="tg-buh4"><span style="color:#333">id</span></td>
    <td class="tg-buh4">Unique identifier representing a bundle (product_id, store_id, date)<br>There is only one id, it does not repeat in the data</td>
    <td class="tg-buh4"><span style="color:#333">int64</span></td>
    <td class="tg-dzk6"><span style="color:#333">0</span></td>
    <td class="tg-buh4"><span style="color:#333">666676</span></td>
  </tr>
  <tr>
    <td class="tg-amwm"><span style="font-weight:bold">2</span></td>
    <td class="tg-0lax"><span style="color:#333">date</span><br></td>
    <td class="tg-0lax"><span style="color:#333">Date of sale</span></td>
    <td class="tg-0lax"><span style="color:#333">object</span></td>
    <td class="tg-baqh"><span style="color:#333">0</span></td>
    <td class="tg-0lax"><span style="color:#333">200</span></td>
  </tr>
  <tr>
    <td class="tg-0vih"><span style="font-weight:bold">3</span></td>
    <td class="tg-buh4"><span style="color:#333">city_name</span></td>
    <td class="tg-buh4"><span style="color:#333">Name of the city where the sale took place</span></td>
    <td class="tg-buh4"><span style="color:#333">object</span></td>
    <td class="tg-dzk6"><span style="color:#333">0</span></td>
    <td class="tg-buh4"><span style="color:#333">10</span></td>
  </tr>
  <tr>
    <td class="tg-amwm"><span style="font-weight:bold">4</span></td>
    <td class="tg-0lax"><span style="color:#333">store_id</span></td>
    <td class="tg-0lax"><span style="color:#333">Unique identifier for each store</span></td>
    <td class="tg-0lax"><span style="color:#333">int64</span></td>
    <td class="tg-baqh"><span style="color:#333">0</span></td>
    <td class="tg-0lax"><span style="color:#333">160</span><br></td>
  </tr>
  <tr>
    <td class="tg-0vih"><span style="font-weight:bold">5</span></td>
    <td class="tg-buh4"><span style="color:#333">category_id</span></td>
    <td class="tg-buh4"><span style="color:#333">Product category</span></td>
    <td class="tg-buh4"><span style="color:#333">int64</span></td>
    <td class="tg-dzk6"><span style="color:#333">0</span></td>
    <td class="tg-buh4"><span style="color:#333">9</span></td>
  </tr>
  <tr>
    <td class="tg-amwm"><span style="font-weight:bold">6</span></td>
    <td class="tg-0lax"><span style="color:#333">product_id</span></td>
    <td class="tg-0lax"><span style="color:#333">Unique identifier for each type of product</span></td>
    <td class="tg-0lax"><span style="color:#333">int64</span></td>
    <td class="tg-baqh"><span style="color:#333">0</span></td>
    <td class="tg-0lax"><span style="color:#333">32</span></td>
  </tr>
  <tr>
    <td class="tg-0vih"><span style="font-weight:bold">7</span></td>
    <td class="tg-buh4"><span style="color:#333">price</span></td>
    <td class="tg-buh4"><span style="color:#333">Price of the product</span></td>
    <td class="tg-buh4"><span style="color:#333">float64</span></td>
    <td class="tg-dzk6"><span style="color:#333">0</span></td>
    <td class="tg-buh4"><span style="color:#333">29</span></td>
  </tr>
  <tr>
    <td class="tg-amwm"><span style="font-weight:bold">8</span></td>
    <td class="tg-0lax"><span style="color:#333">weather_desc</span></td>
    <td class="tg-0lax"><span style="color:#333">Weather description</span></td>
    <td class="tg-0lax"><span style="color:#333">object</span></td>
    <td class="tg-baqh"><span style="color:#333">0</span></td>
    <td class="tg-0lax"><span style="color:#333">16</span></td>
  </tr>
  <tr>
    <td class="tg-0vih"><span style="font-weight:bold;color:#333">9</span></td>
    <td class="tg-buh4"><span style="color:#333">humidity</span></td>
    <td class="tg-buh4"><span style="color:#333">Humidity in the city on the day of sale</span></td>
    <td class="tg-buh4"><span style="color:#333">float64</span></td>
    <td class="tg-dzk6"><span style="color:#333">0</span></td>
    <td class="tg-buh4"><span style="color:#333">916</span></td>
  </tr>
  <tr>
    <td class="tg-amwm"><span style="font-weight:bold;color:#333">10</span></td>
    <td class="tg-0lax"><span style="color:#333">temperature</span></td>
    <td class="tg-0lax"><span style="color:#333">Temperature in the city on the day of sale</span></td>
    <td class="tg-0lax"><span style="color:#333">float64</span></td>
    <td class="tg-baqh"><span style="color:#333">0</span></td>
    <td class="tg-0lax"><span style="color:#333">505</span></td>
  </tr>
  <tr>
    <td class="tg-0vih"><span style="font-weight:bold;color:#333">11</span></td>
    <td class="tg-buh4"><span style="color:#333">pressure</span></td>
    <td class="tg-buh4"><span style="color:#333">Atmosphere pressure in the city on the day of sale</span></td>
    <td class="tg-buh4"><span style="color:#333">float64</span></td>
    <td class="tg-dzk6"><span style="color:#333">0</span></td>
    <td class="tg-buh4"><span style="color:#333">344</span></td>
  </tr>
  <tr>
    <td class="tg-amwm"><span style="font-weight:bold;color:#333">12</span></td>
    <td class="tg-0lax"><span style="color:#333">sales</span></td>
    <td class="tg-0lax"><span style="color:#333">Number of product sales (this is what I should predict)</span></td>
    <td class="tg-0lax"><span style="color:#333">int64</span></td>
    <td class="tg-baqh"><span style="color:#333">0</span></td>
    <td class="tg-0lax"><span style="color:#333">249</span></td>
  </tr>
</tbody>
</table></center>


There are 666 676 observations in the train dataframe and 24 836 observations in the test dataframe.  <br>
The train dataframe describes data in 7 months period, from June 29, 2021 to February 13, 2022. <br>
The test dataframe describes data in one week period from February 14, 2022 to February 20, 2022.  <br>
The dataframe contains information from 10 cities:  
Moscow, 	St. Petersburg, Krasnodar,	Samara, Nizhny Novgorod	Rostov-on-Don, Volgograd,	Voronezh, Kazan, and	Yekaterinburg.

<div align="justify">The table presented describes columns with numeric data. Further, the table below shows the Pearson correlation between sales column and other numeric columns.</div>

In [ ]:
# # the code is represented in the table below
# train.iloc[:, [6,8,9, 10,11]].describe().loc[['mean','min', 'max', '25%', '50%', '75%']].round(1)

<center><p>Statistical description of numeric data</p>
  
<style type="text/css">
.tg  {border:none;border-collapse:collapse;border-color:#ccc;border-spacing:0;}
.tg td{background-color:#fff;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#f0f0f0;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{border-color:inherit;font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-zwlc{background-color:#f9f9f9;border-color:inherit;font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-abip{background-color:#f9f9f9;border-color:inherit;text-align:center;vertical-align:top}
@media screen and (max-width: 767px) {.tg {width: auto !important;}.tg col {width: auto !important;}.tg-wrap {overflow-x: auto;-webkit-overflow-scrolling: touch;}}</style>
<div class="tg-wrap"><table class="tg">
<thead>
  <tr>
    <th class="tg-c3ow"></th>
    <th class="tg-7btt"><span style="font-weight:bold">price</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">humidity</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">temperature</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">pressure</span></th>
    <th class="tg-7btt"><span style="font-weight:bold">sales</span></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-zwlc"><span style="font-weight:bold">mean</span></td>
    <td class="tg-abip">5.1</td>
    <td class="tg-abip">74.3</td>
    <td class="tg-abip">4.9</td>
    <td class="tg-abip">751</td>
    <td class="tg-abip">10</td>
  </tr>
  <tr>
    <td class="tg-7btt"><span style="font-weight:bold">min</span></td>
    <td class="tg-c3ow">1.9</td>
    <td class="tg-c3ow">13.8</td>
    <td class="tg-c3ow">-24.0</td>
    <td class="tg-c3ow">710</td>
    <td class="tg-c3ow">0</td>
  </tr>
  <tr>
    <td class="tg-zwlc"><span style="font-weight:bold">max</span></td>
    <td class="tg-abip">18.6</td>
    <td class="tg-abip">100.0</td>
    <td class="tg-abip">34.3</td>
    <td class="tg-abip">779</td>
    <td class="tg-abip">275</td>
  </tr>
  <tr>
    <td class="tg-7btt"><span style="font-weight:bold">25%</span></td>
    <td class="tg-c3ow">3.0</td>
    <td class="tg-c3ow">59.8</td>
    <td class="tg-c3ow">-3.3</td>
    <td class="tg-c3ow">745</td>
    <td class="tg-c3ow">2</td>
  </tr>
  <tr>
    <td class="tg-zwlc"><span style="font-weight:bold">50%</span></td>
    <td class="tg-abip">4.1</td>
    <td class="tg-abip">79.7</td>
    <td class="tg-abip">4.4</td>
    <td class="tg-abip">751</td>
    <td class="tg-abip">5</td>
  </tr>
  <tr>
    <td class="tg-7btt"><span style="font-weight:bold">75%</span></td>
    <td class="tg-c3ow">6.0</td>
    <td class="tg-c3ow">92.4</td>
    <td class="tg-c3ow">11.8</td>
    <td class="tg-c3ow">758</td>
    <td class="tg-c3ow">12</td>
  </tr>
</tbody>
</table></div></center>


In [ ]:
# # the code is represented in the table below
# corr1 = train.corrwith(train['sales'])
# corr1  = pd.DataFrame(corr1, columns = ['sales'])
# corr1 = corr1.sort_values('sales',ascending=False)
# corr1.style.background_gradient(cmap='coolwarm').set_precision(2)

<center><p>Pearson correlation between sales and other columns</p>
  
<style type="text/css">
.tg  {border:none;border-collapse:collapse;border-color:#ccc;border-spacing:0;}
.tg td{background-color:#fff;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#f0f0f0;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-kug9{background-color:#94B6FF;border-color:inherit;color:#D5D5D5;text-align:center;vertical-align:top}
.tg .tg-p85h{background-color:#5572DF;border-color:inherit;color:#F1F1F1;text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-xr90{background-color:#5D7CE6;border-color:inherit;color:#F1F1F1;text-align:center;vertical-align:top}
.tg .tg-7btt{border-color:inherit;font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-abip{background-color:#f9f9f9;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-bd2j{background-color:#B40426;border-color:inherit;color:#F1F1F1;text-align:center;vertical-align:top}
.tg .tg-8jpj{background-color:#93B5FE;border-color:inherit;color:#D5D5D5;text-align:center;vertical-align:top}
.tg .tg-bdi0{background-color:#5875E1;border-color:inherit;color:#F1F1F1;text-align:center;vertical-align:top}
.tg .tg-3u24{background-color:#5470DE;border-color:inherit;color:#F1F1F1;text-align:center;vertical-align:top}
.tg .tg-9enl{background-color:#4E68D8;border-color:inherit;color:#F1F1F1;text-align:center;vertical-align:top}
.tg .tg-0cd6{background-color:#3B4CC0;border-color:inherit;color:#F1F1F1;text-align:center;vertical-align:top}
@media screen and (max-width: 767px) {.tg {width: auto !important;}.tg col {width: auto !important;}.tg-wrap {overflow-x: auto;-webkit-overflow-scrolling: touch;}}</style>
<div class="tg-wrap"><table class="tg">
<thead>
  <tr>
    <th class="tg-7btt">Column name</th>
    <th class="tg-7btt"><span style="font-weight:bold">sales</span></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-abip">sales</td>
    <td class="tg-bd2j"><span style="color:#F1F1F1;background-color:#B40426">1.00</span></td>
  </tr>
  <tr>
    <td class="tg-c3ow">product_id</td>
    <td class="tg-kug9"><span style="color:#000;background-color:#94B6FF">0.14</span></td>
  </tr>
  <tr>
    <td class="tg-abip">humidity</td>
    <td class="tg-8jpj"><span style="color:#000;background-color:#93B5FE">0.13</span></td>
  </tr>
  <tr>
    <td class="tg-c3ow">pressure</td>
    <td class="tg-xr90"><span style="color:#F1F1F1;background-color:#5D7CE6">-0.06</span></td>
  </tr>
  <tr>
    <td class="tg-abip">temperature</td>
    <td class="tg-bdi0"><span style="color:#F1F1F1;background-color:#5875E1">-0.07</span></td>
  </tr>
  <tr>
    <td class="tg-c3ow">id</td>
    <td class="tg-p85h"><span style="color:#F1F1F1;background-color:#5572DF">-0.08</span></td>
  </tr>
  <tr>
    <td class="tg-abip">store_id</td>
    <td class="tg-3u24"><span style="color:#F1F1F1;background-color:#5470DE">-0.09</span></td>
  </tr>
  <tr>
    <td class="tg-c3ow">category_id</td>
    <td class="tg-9enl"><span style="color:#F1F1F1;background-color:#4E68D8">-0.11</span></td>
  </tr>
  <tr>
    <td class="tg-abip">price</td>
    <td class="tg-0cd6"><span style="color:#F1F1F1;background-color:#3B4CC0">-0.19</span></td>
  </tr>
</tbody>
</table></div></center>

<h3 id="Data-visualisation">Data visualisation</h3> 

<div align="justify">All of the graphs presented in this work are interactive. It is possible to choose one out of many lines shown on the graph and choose the interval across both axes to explore the data. To go back to initial state of the graph it is necessary to double-click the graph or click "Reset axes" button at the top right corner. 
<br><br>
It seems intuitive to start looking for patterns with data exploration of how the cumulative number of sales changes each day in each city.
The graph below shows 7-months period in 10 cities. There are several features that are evident:
<li>First feature that can be seen in the graph is that the number of sales noticably changes throughout the week in each city. </li>
<li>Second thing is that in different cities there is a different rate of sales. In larger cities, like Moscow, the sales rate is the highest. </li>
<li>Third, the noticeable sharp decrease in sales can be observed around New Year, which is an important national holiday in Russia, during which people generally stay at home with their families and most of the stores are closed. This effect cannot be considered as an outlier and should be extrapolated to other major public holidays. However, no such holidays are present in February.</li>
<li>In the end, the tendency can be noticed that the total number of sales is started to rise in the winter. </li></div>

In [ ]:
# Sales in every city for 7 month period

# subset the data
group = train[['date', 'city_name', 'sales']].groupby(['date', 'city_name'], as_index=False).sum()

# plot graph
fig = px.line(
              group, 
              x="date", 
              y="sales", 
              color='city_name'
              )

# modify layout of the graph 
fig.update_layout(
                  font_family="'Nunito', sans-serif",
                  title={'text': "Sales in every city for 7 month period", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  xaxis_title='Date',
                  yaxis_title='Sales',
                  legend=dict(y=0.5, title="City")
                  )
fig.show() 

<div align="justify">The next step is to observe another time dependency, but with aggragted number of sales in each month. On the graph below it can be seen that in every city the total number of sales slowly droped at the beggining of autumn and then started to grow again at the beggining of winter. On this graph the difference between total number of sales in different cities is much clearer. This fact is not unusual, since the cities in Russia are quite disproportionate in population size and salary levels (supplementary information is shown in the table below). Information about population size and salary levels was found <a href="https://top-rf.ru/places/335-goroda-rossii.html">here</a> and <a href="https://gogov.ru/articles/average-salary">here</a>.</div>

In [ ]:
# Total monthly sales in every city

# subset the data
group = train[['date', 'city_name', 'sales']]
group.date = pd.to_datetime(group.date)  
group = group[(group.date.dt.month != 2) & (group.date.dt.month != 7)] # subset data - so that I compare whole months 
group = group.groupby([group.date.dt.month, group.city_name], as_index=False).sum()
group["month"] = np.repeat(pd.to_datetime(train.date[(pd.to_datetime(train.date).dt.month != 2) & \
                                                     (pd.to_datetime(train.date).dt.month != 7)]).dt.strftime("%B").unique(), 10)
# plot graph
fig = px.line(
              group, 
              x="month", 
              y="sales", 
              color='city_name'
              )

# modify layout of the graph 
fig.update_layout(
                  font_family="Nunito, sans-serif",
                  title={'text': "Total monthly sales in every city", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  xaxis_title='Month',
                  yaxis_title='Sales',
                  legend=dict(y=0.5, title="City"), 
                  )
fig.show() 

<center><p>Population and salary levels in cities</p>

<style type="text/css">
.tg  {border:none;border-collapse:collapse;border-color:#ccc;border-spacing:0;}
.tg td{background-color:#fff;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family: Nunito, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#f0f0f0;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family: Nunito, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-dzk6{background-color:#f9f9f9;text-align:center;vertical-align:top}
.tg-sort-header::-moz-selection{background:0 0}
.tg-sort-header::selection{background:0 0}.tg-sort-header{cursor:pointer}
.tg-sort-header:after{content:'';float:right;margin-top:7px;border-width:0 5px 5px;border-style:solid;
  border-color:#404040 transparent;visibility:hidden}
.tg-sort-header:hover:after{visibility:visible}
.tg-sort-asc:after,.tg-sort-asc:hover:after,.tg-sort-desc:after{visibility:visible;opacity:.4}
.tg-sort-desc:after{border-bottom:none;border-width:5px 5px 0}@media screen and (max-width: 767px) {.tg {width: auto !important;}.tg col {width: auto !important;}.tg-wrap {overflow-x: auto;-webkit-overflow-scrolling: touch;}}</style>
<div class="tg-wrap"><table id="tg-vqJLh" class="tg">
<thead>
  <tr>
    <th class="tg-amwm">№</th>
    <th class="tg-amwm">City</th>
    <th class="tg-amwm">Population, M</th>
    <th class="tg-amwm">Salary, K</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-dzk6">1</td>
    <td class="tg-dzk6">Moscow</td>
    <td class="tg-dzk6">12,66</td>
    <td class="tg-dzk6">111,1</td>
  </tr>
  <tr>
    <td class="tg-baqh">2</td>
    <td class="tg-baqh">St. Petersburg</td>
    <td class="tg-baqh">5,38</td>
    <td class="tg-baqh">76,0</td>
  </tr>
  <tr>
    <td class="tg-dzk6">3</td>
    <td class="tg-dzk6"><span style="color:#333">Krasnodar</span></td>
    <td class="tg-dzk6">0,95</td>
    <td class="tg-dzk6">40,8</td>
  </tr>
  <tr>
    <td class="tg-baqh">4</td>
    <td class="tg-baqh"><span style="color:#333">Nizhny Novgorod</span></td>
    <td class="tg-baqh">1,26</td>
    <td class="tg-baqh">41,5</td>
  </tr>
  <tr>
    <td class="tg-dzk6">5</td>
    <td class="tg-dzk6"><span style="color:#333">Volgograd</span></td>
    <td class="tg-dzk6">1,00</td>
    <td class="tg-dzk6">38,1</td>
  </tr>
  <tr>
    <td class="tg-baqh">6</td>
    <td class="tg-baqh"><span style="color:#333">Kazan</span></td>
    <td class="tg-baqh">1,26</td>
    <td class="tg-baqh">44,9</td>
  </tr>
  <tr>
    <td class="tg-dzk6">7</td>
    <td class="tg-dzk6">Samara</td>
    <td class="tg-dzk6">1,14</td>
    <td class="tg-dzk6">42,9</td>
  </tr>
  <tr>
    <td class="tg-baqh">8</td>
    <td class="tg-baqh">Rostov-on-Don</td>
    <td class="tg-baqh">1,14</td>
    <td class="tg-baqh">39,1</td>
  </tr>
  <tr>
    <td class="tg-dzk6">9</td>
    <td class="tg-dzk6">Voronezh</td>
    <td class="tg-dzk6">1,05</td>
    <td class="tg-dzk6">40,9</td>
  </tr>
  <tr>
    <td class="tg-baqh">10</td>
    <td class="tg-baqh">Yekaterinburg</td>
    <td class="tg-baqh">1,50</td>
    <td class="tg-baqh">48,4</td>
  </tr>
</tbody>
</table></div></center>

<div align="justify">
Since the number of sales drastically changes from sity to city, it is interesting to see the difference between mean daily sales in each city. Moreover, the gap between mean number of sales each day of the week should be large. It is important to check if there is any strong correlation in price vs sales and weather vs sales.

In the set of graphs below it can be seen that mean of sales is low at the beginning of the week and then the sales rise up by the weekend. The dependence is weak, but noticable, so it is reasonable to add additinal variable "Day of the week" to the dataframe and fill it with number of the day of the week (from 1 to 7). Moreover, another variable "Weekend" was added with values of 0 and 1, showing if the day is weekend or not. 
 
The rate of sales of products varies depending on their price. There are a lot of products with low prices. That means that the cheaper the product the more often it gets selled. 

On the left bottom side of the graph dependence can be observed in mean daily sales in each city. The column of city names is converted to numeric format with "one-hot encoding" approach since it has substansive correlation with sales.

Lastly, there is a weak dependence between sales and weather conditions. Consequently, conversion to numeric format with "one-hot encoding" approach was undertaken. Although it is hard to interpret this dependence properly, and moreover some weather types are more frequent than others in different cities.</div>

In [ ]:
# subset data for plot "Mean sales for every day of the week"
weekdays = train[['date', 'sales']]
weekdays.date = pd.to_datetime(weekdays.date)  
weekdays = weekdays.groupby([weekdays.date.dt.weekday], as_index=False).mean()
weekdays["weekdays"] = pd.to_datetime(train.iloc[4:].date).dt.strftime("%A").unique() # add weekdays from monday to sunday

# subset data for plot "Histogram of sales by prices"
prices = train[['price', 'sales']]
prices = prices.groupby(prices.price, as_index=False).count()


# subset data for plot "Mean number of sales depending on weather"
mean_weather = train[['weather_desc', 'sales']]
mean_weather = mean_weather.replace(       
                                    (list(mean_weather.weather_desc.unique())),
                                    list(range(1,17)))
mean_weather = mean_weather.groupby([mean_weather.weather_desc], as_index=False).mean()
mean_weather.sort_values('sales',ascending=False, inplace=True)
mean_weather = mean_weather.replace(         
                                    list(range(1,17)),  
                                    list(train.weather_desc.unique()))

# subset data for plot "Mean daily sales in each city"
daily = train[['city_name', 'sales', 'date']]
daily.date = pd.to_datetime(daily.date)  
daily = daily.replace(       
                      ("Moscow", "St.Petersburg", "Krasnodar", "Samara", "Nizhny.Novgorod", "Rostov-on-Don", \
                                                            "Volgograd", "Voronezh", "Kazan", "Yekaterinburg"),
                      (1,2,3,4,5,6,7,8,9,10)  )
daily.city_name = daily.city_name.astype('int')
daily = daily.groupby([daily.city_name, daily.date], as_index=False).sum()
daily = daily.groupby([daily.city_name], as_index=False).mean()
daily.sort_values('sales',ascending=False, inplace=True)
daily = daily.replace(         
                      (1,2,3,4,5,6,7,8,9,10),  
                      ("Moscow", "St.Petersburg", "Krasnodar", "Samara", "Nizhny.Novgorod", "Rostov-on-Don", \
                                                            "Volgograd", "Voronezh", "Kazan", "Yekaterinburg"))

# plot (4 subplots)
figure = make_subplots(
                       rows=2, 
                       cols=2, 
                       subplot_titles=("Mean sales for every day of the week", 
                                       "Histogram of sales by prices", 
                                       "Mean daily sales in each city", 
                                       "Mean number of sales depending on weather"),
                       horizontal_spacing = 0.07, 
                       )


# 1.plot - Mean sales for every day of the week
figure.add_trace(go.Bar(
                        x=weekdays.weekdays, 
                        y=weekdays.sales
                        ), 
                 row=1, 
                 col=1
                 )
figure.update_xaxes(title_text='Day of the week', row=1, col=1)
figure.update_yaxes(title_text='Sales', row=1, col=1)

# 2.plot - Histogram of sales by prices
figure.add_trace(go.Bar(
                        x=prices.price, 
                        y=prices.sales,  
                        width=0.3
                        ), 
                 row=1, 
                 col=2
                 )
figure.update_xaxes(title_text='Price', row=1, col=2)
figure.update_yaxes(title_text='Sales', row=1, col=2)

# 3.plot - Mean number of sales depending on weather
figure.add_trace(go.Bar(
                        x=daily.city_name, 
                        y=daily.sales
                        ),
                 row=2, 
                 col=1
                 )
figure.update_xaxes(title_text='City', row=2, col=1)
figure.update_yaxes(title_text='Sales', row=2, col=1)

# 4.plot - Mean daily sales in each city
figure.add_trace(go.Bar(
                        x=mean_weather.weather_desc, 
                        y=mean_weather.sales
                        ),
                 row=2, 
                 col=2
                 )
figure.update_xaxes(title_text='Weather', row=2, col=2)
figure.update_yaxes(title_text='Sales', row=2, col=2)
fig.update_xaxes(row=2, col=2, tickangle=45) 

# modify layout of the graph 
figure.update_layout(
                    height=800, 
                    font_family="'Nunito', sans-serif",
                    title={'text': "Correlations between sales and other columns", 'y':0.99, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                    showlegend=False, 
                    margin=dict(l=40, r=10, t=60, b=60),
                    )

figure.show()

<div align="justify">To build predictions of sales it is necessary to find data that correlates strongly with the number of sales. For that several lag features were added to the dataframe. Three columns were added with sales information from three previous weeks for each product from certain store. Another column was added with the average value of aforecited variables. Furthermore, mean of all-time sales of the product for a certain day of the week in each city for each store was added. Correlation matrix was obtained for the new dataframe.

New columns show high correlation with the number of sales and should significantly improve predictions of future sales.</div>

In [ ]:
# concat both data sets
group = pd.concat([train,test], axis=0)  

group.date = pd.to_datetime(group.date)                              # convert date column to date format    
group = group.assign(dayofweek=group.date.dt.dayofweek)                 # create day of week column
group = group.assign(weekend=lambda x: 1*(group.date.dt.dayofweek>4))   # create weekend column
group.date = pd.to_datetime(group.date)  

# create group1 for sorting cities in the order of higher number of daily sales
group1 = group.groupby([group.city_name, group.date], as_index=False).sum()
group1 = group1.groupby([group1.city_name], as_index=False).mean()
group1.sort_values('sales',ascending=False, inplace=True)
cities = list(group1.city_name.unique())
# convert city names to numeric format
group = group.replace(
    cities,
    list(range(10, 0, -1))  )

# create group2 for sorting weather_desc in the order of higher number of sales during ceratin weather
group2 = group.replace(       
    (list(group.weather_desc.unique())),
    list(range(1,17)))

group2 = group2.groupby([group2.weather_desc], as_index=False).mean()
group2.sort_values('sales',ascending=False, inplace=True)
group2 = group2.replace(         
    list(range(1,17)),  
    list(train.weather_desc.unique()))
weather = list(group2.weather_desc.unique())
# convert weather to numeric format
group = group.replace(       
    weather,
    list(range(16,0, -1)))

# Add mean number of sales for each day of the week for each type of product in each store
group_2 = group.groupby(['product_id', 'store_id', 'city_name', 'dayofweek'])[['sales']].mean().reset_index()
group_2.rename(columns={'sales':'day_product_mean'}, inplace=True)
group = pd.merge(group, group_2, how="left", on=['product_id', 'store_id', 'dayofweek', 'city_name'])

# add lag_day_7-15 which shows lag of sales for product_id from store_id 1-3 weeks ago
group_3 = group.groupby(['product_id', 'store_id', 'date'])[['sales']].sum().reset_index()
for i in [7, 14, 21]:
    group_3[f'lag_day_{i}'] = group_3['sales'].shift(i)                                # add lags for 1-3 weeks
group_3['lag_days_mean'] = (group_3['lag_day_7'] + group_3['lag_day_14'] + group_3['lag_day_21'])/3.  # add mean of lags
group_3.drop(['sales'], axis=1, inplace=True)
group_3.dropna(inplace=True)                                                         # drop created NAs from first 3 weeks
#  merge lag_day_{i} and lag_days_mean columns with initial dataframe
group = pd.merge(group, group_3, how="left", on=['product_id', 'store_id', 'date'])        # first three weeks will have NA b/c no lags exist for them

# Build correlations of columns with sales column
corr1 = group.corrwith(group['sales'])
corr1  = pd.DataFrame(corr1, columns = ['sales'])
corr1 = corr1.sort_values('sales',ascending=False)

# correlation table; it is visualised below
# corr1.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
# shoose random 1000 samples of data to show; 
# so that that the program runs fast and file is not too heavy
sample = group.sample(1000)
sales = sample.sales
sample.drop(['date', 'weekend', 'sales'], axis=1, inplace=True)

# plot graph
fig = make_subplots(
                    rows=4, 
                    cols=6, 
                    horizontal_spacing = 0.005, 
                    shared_yaxes=True, 
                    column_widths=[0.12, 0.04, 0.21, 0.21, 0.21, 0.21],
                    specs=[
                           [{'rowspan': 4}, {}, {}, {}, {}, {} ],
                           [None,           {}, {}, {}, {}, {} ],
                           [None,           {}, {}, {}, {}, {} ],
                           [None,           {}, {}, {}, {}, {} ] 
                           ])

# add correlation table heatmap
corr1 = corr1.round(2)
corr1 = corr1.sort_values('sales',ascending=True)
trace1 = go.Heatmap(
                    x0=0, 
                    z=corr1, 
                    showscale=False, 
                    y=list(corr1.index), 
                    textfont={"size":10},
                    texttemplate='%{z}', 
                    colorscale = 'magma',  
                    ) 

# modify layout of the correlation table 
fig.add_trace( trace1, row=1, col=1)
fig.update_traces(col=5, colorscale = 'magma') 
fig.update_xaxes(row=1, col=1, visible=False, showticklabels=False)

# add matrix of graphs that show how columns change with the sales column
index=0
names = sample.columns
for i in [3,4,5,6]:       # cols
    for j in [1,2,3,4]:   # rows
        
        fig.add_trace(go.Scatter(
                                 x=list(sample.iloc[:, index]), 
                                 y=list(sales),
                                 marker=dict(color="crimson", size=4, opacity=0.5), 
                                 mode="markers"
                                 ), 
                      row=j, 
                      col=i)
        
        fig.update_xaxes(title_text=names[index], row=j, col=i)
        if i == 3:
            fig.update_yaxes(title_text="sales", row=j, col=i)
        index+=1

# modify layout of the graph 
fig.update_layout(
                  height=800, 
                  font_family="'Nunito', sans-serif",
                  title={'text': "Correlations between sales and other columns", 'y':0.97, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  showlegend=False, 
                  margin=dict(l=40, r=10, t=60, b=60), 
                  )

fig.show()

<div align="justify">For further evaluation of ML models mean absolute error (MAE) is used as quality metric. MAE shows how many times the forecast is wrong on average. This metric is very easy to interpret. For example, metric value equal to 5 means that the model is on average wrong by 5. The metric cannot be negative, since absolute values of errors are taken. For an ideal model this metric will be equal to 0. The metric is not sensitive to outliers.  </div>

$$MAE = \frac1N \sum ^{N}_{i=1} |y_i-\hat y_i|$$

<br> 
<h2 id="Building-ML-models">Building ML models</h2>  

--- 

<div align="justify">Two dataframes were obtained (df_train, df_test), with all the modifications described previuosly. First three weeks in df_train were droped, since they have NAs in lag feature columns. First, several ML models were trained on df_train and the best model with minimal MAE score was chosen. For this, df_train was divided into 4 parts: y_train, x_train, y_test, x_test. <br>
The ML models that were used in the following work are listed below. For each model, MAE scores were calculated.
</div>

In [ ]:
# I take premodified in the previous section concatenated dataframe;
# Divide it into test and train dataframes
# Divide each dataframe into x and y data
# Drop all the NAs, which were created in the previous steps

# drop first three weeks since they have NAs
group.drop(index=group.index[:21], axis=0, inplace=True)  

# extract sales column -> y data
sales = group.sales.dropna()   

# drop column with date format and column with y data;
# divide concatenated dataframe into train and test
df_test = group[group.id > 666676].drop(labels=["sales", "date"], axis=1, inplace=False)  
df_train = group[group.id < 666677].drop(labels=["sales", "date"], axis=1, inplace=False)

# make 4 subsets for training and testing
X_train, X_test, y_train, y_test = train_test_split(
                                                    df_train,
                                                    sales,
                                                    train_size = 0.9999, 
                                                    test_size = 0.0001, 
                                                    random_state=1984
                                                    )


  ML models:

- k-nearest neighbors (KNN) 
- Linear Regression
- Decision Tree Regression
- Ensemble models
  - Random forest
  - Gradient Boosting
  - Mean of all regressors

In [ ]:
# Linear Regression model

model = LinearRegression()
model.fit(X_train, y_train)

forecast_lm = model.predict(X_test)
mae = mean_absolute_error(y_test, forecast_lm)

# print(f"Linear Regression model: MAE = {mae:.2f} < 4.10 " if mae < 4.10 else f"Linear model: MAE = {mae:.2f} > 4.10 ")

# combine predictions with date column and sort by date
prediction_lm = pd.merge(X_test, group[['id', 'date']], how="left", on=['id'])
prediction_lm.sort_values('date',ascending=True, inplace=True)

# make plot
fig_lm = go.Figure([
                    go.Scatter(
                               x=prediction_lm.date, 
                               y=y_test, 
                               name='Factual data', 
                               mode='lines+markers', 
                               marker_color='rgba(0, 0, 0, .6)', 
                               showlegend=False
                               ),
                    go.Scatter(
                               x=prediction_lm.date, 
                               y=forecast_lm, 
                               name='Linear Regression', 
                               mode='lines+markers'
                               )
                    ], 
                    layout=go.Layout(
                                     title=go.layout.Title( text=f"Linear Regression model: MAE = {mae:.2f} < 4.10 " \
                                                           if mae < 4.10 else f"Linear model: MAE = {mae:.2f} > 4.10 ", 
                                                           ) 
                                     )
                   )
# fig_lm.show()

In [ ]:
# DecisionTreeRegressor model 
tree_clf = DecisionTreeRegressor(max_depth=12).fit(X_train, y_train)
forecast_tree = tree_clf.predict(X_test)
mae = mean_absolute_error(y_test, forecast_tree)
# print(f"TreeClassifier model: MAE = {mae:.2f} < 4.10 " if mae < 4.10 else f"TreeClassifier model: MAE = {mae:.2f} > 4.10 ")

# combine predictions with date column and sort by date
prediction_tree = pd.merge(X_test, group[['id', 'date']], how="left", on=['id'])
prediction_tree.sort_values('date',ascending=True, inplace=True)

fig_tree = go.Figure([
                      go.Scatter(
                                 x=prediction_tree.date, 
                                 y=y_test, 
                                 name='DFEDTr',
                                 mode='lines+markers',
                                 marker_color='rgba(0, 0, 0, .6)', 
                                 showlegend=False
                                 ),
                      go.Scatter(
                                 x=prediction_tree.date, 
                                 y=forecast_tree, 
                                 name='Decision Tree',
                                 mode='lines+markers'
                                 )
                      ], 
                      layout=go.Layout(
                                       title=go.layout.Title(text=f"TreeClassifier model: MAE = {mae:.2f} < 4.10 " \
                                                             if mae < 4.10 else f"Linear model: MAE = {mae:.2f} > 4.10 "
                                                             )  
                                       )
                      )
# fig_tree.show()

In [ ]:
# KNN model
# have to choose perfect number of neighbours
knn_clf = KNeighborsRegressor(n_neighbors=12)
knn_clf.fit(X_train, y_train)
forecast_knn = knn_clf.predict(X_test)
mae = mean_absolute_error(y_test, forecast_knn)
# print(f"KNN model: MAE = {mae:.2f} < 4.10 " if mae < 4.10 else f"KNN model: MAE = {mae:.2f} > 4.10 ")

# combine predictions with date column and sort by date
prediction_knn = pd.merge(X_test, group[['id', 'date']], how="left", on=['id'])
prediction_knn.sort_values('date',ascending=True, inplace=True)

fig_knn = go.Figure([
                     go.Scatter(
                                x=prediction_knn.date, 
                                y=y_test, 
                                name='fact', 
                                mode='lines+markers', 
                                marker_color='rgba(0, 0, 0, .6)', 
                                showlegend=False
                                ),
                     go.Scatter(
                                x=prediction_knn.date, 
                                y=forecast_knn, 
                                name='KNN', 
                                ode='lines+markers'
                                )
                     ], 
                     layout=go.Layout(
                                      title=go.layout.Title(text=f"KNN model: MAE = {mae:.2f} < 4.10 " if mae < 4.10 \
                                                            else f"Linear model: MAE = {mae:.2f} > 4.10 "
                                                            )  
                                      )
                     )
# fig_knn.show()

In [ ]:
# RandomForestRegressor model
rf_model = RandomForestRegressor(n_estimators = 10, 
                                #  min_samples_split=250
                                 )
rf_model.fit(X_train, y_train)
rf_forecast = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, rf_forecast)
# print(f"RandomForestRegressor model: MAE = {mae:.2f} < 4.10 " if mae < 4.10 else f"KNN model: MAE = {mae:.2f} > 4.10 ")

# combine predictions with date column and sort by date
prediction_rfc = pd.merge(X_test, group[['id', 'date']], how="left", on=['id'])
prediction_rfc.sort_values('date',ascending=True, inplace=True)

fig_rcf = go.Figure([
                     go.Scatter(
                                x=prediction_rfc.date, 
                                y=y_test, 
                                name='fact', 
                                mode='lines+markers', 
                                marker_color='rgba(0, 0, 0, .6)', 
                                showlegend=False
                                ),
                     go.Scatter(
                                x=prediction_rfc.date, 
                                y=rf_forecast, 
                                name='Random Forest', 
                                mode='lines+markers')
                     ], 
                     layout=go.Layout(
                                      title=go.layout.Title(text=f"RandomForestRegressor: MAE = {mae:.2f} < 4.10 " \
                                                            if mae < 4.10 else f"Linear model: MAE = {mae:.2f} > 4.10 "
                                                            )  
                                      )
                    )
# fig_rcf.show()

In [ ]:
# GradientBoostingRegressor model
gb_model = GradientBoostingRegressor(loss='absolute_error', n_estimators=50,
                                 )
gb_model.fit(X_train, y_train)
gb_forecast = gb_model.predict(X_test)
mae = mean_absolute_error(y_test, gb_forecast)
# print(f"GradientBoostingRegressor model: MAE = {mae:.2f} < 4.10 " if mae < 4.10 else f"KNN model: MAE = {mae:.2f} > 4.10 ")

# combine predictions with date column and sort by date
prediction_gb = pd.merge(X_test, group[['id', 'date']], how="left", on=['id'])
prediction_gb.sort_values('date', ascending=True, inplace=True)

fig_gb = go.Figure([
                    go.Scatter(
                               x=prediction_gb.date, 
                               y=y_test, 
                               name='fact', 
                               mode='lines+markers', 
                               marker_color='rgba(0, 0, 0, .6)', 
                               showlegend=False
                               ),
                    go.Scatter(
                               x=prediction_gb.date, 
                               y=gb_forecast, 
                               name='Gradient Boosting', 
                               mode='lines+markers')
                    ], 
                    layout=go.Layout(
                                 title=go.layout.Title(text=f"GradientBoostingRegressor: MAE = {mae:.2f} < 4.10 " \
                                                       if mae < 4.10 else f"Linear model: MAE = {mae:.2f} > 4.10 "
                                                       )  
                                     )
                  )
# fig_gb.show()

In [ ]:
# Mean of all regressors
mean_of_voices = (forecast_knn + forecast_tree + forecast_lm + gb_forecast + rf_forecast)/5
mae = mean_absolute_error(y_test, mean_of_voices)
# print(f"Ensemble model: MAE = {mae:.2f} < 4.10 " if mae < 4.10 else f"Ensemble model: MAE = {mae:.2f} > 4.10 ")

# combine predictions with date column and sort by date
prediction_ensemble = pd.merge(X_test, group[['id', 'date']], how="left", on=['id'])
prediction_ensemble.sort_values('date',ascending=True, inplace=True)

fig_all = go.Figure([
                     go.Scatter(
                                x=prediction_ensemble.date, 
                                y=y_test, 
                                name='Factual data', 
                                mode='lines+markers', 
                                marker_color='rgba(0, 0, 0, .6)'
                                ),
                     go.Scatter(
                                x=prediction_ensemble.date, 
                                y=mean_of_voices, 
                                name='Mean of all regressors', 
                                mode='lines+markers', 
                                marker_color='rgb(60, 150, 60)'
                                )
                     ], 
                     layout=go.Layout(
                                      title=go.layout.Title(text=f"Mean of all regressors: MAE = {mae:.2f} < 4.10 " \
                                                            if mae < 4.10 else f"Linear model: MAE = {mae:.2f} > 4.10 "
                                                            )  
                                      )
                    )
# fig_all.show()

<div align="justify">MAE scores of used ML models are presented in the table below. Predictions and factual data are visualised in the graph. It can be seen that Decision Tree model has the best predictions and that is supported by minimal MAE score.
</div>

<center>
<style type="text/css">
.tg  {border:none;border-collapse:collapse;border-color:#ccc;border-spacing:0;}
.tg td{background-color:#fff;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#f0f0f0;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-dzk6{background-color:#f9f9f9;text-align:center;vertical-align:top}
.tg-sort-header::-moz-selection{background:0 0}
.tg-sort-header::selection{background:0 0}.tg-sort-header{cursor:pointer}
.tg-sort-header:after{content:'';float:right;margin-top:7px;border-width:0 5px 5px;border-style:solid;
  border-color:#404040 transparent;visibility:hidden}
.tg-sort-header:hover:after{visibility:visible}
.tg-sort-asc:after,.tg-sort-asc:hover:after,.tg-sort-desc:after{visibility:visible;opacity:.4}
.tg-sort-desc:after{border-bottom:none;border-width:5px 5px 0}@media screen and (max-width: 767px) {.tg {width: auto !important;}.tg col {width: auto !important;}.tg-wrap {overflow-x: auto;-webkit-overflow-scrolling: touch;}}</style>
<div class="tg-wrap"><table id="tg-0bXqe" class="tg">
<thead>
  <tr>
    <th class="tg-amwm">Model</th>
    <th class="tg-amwm"><span style="font-weight:bold">MAE</span></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-dzk6">Decision Tree</td>
    <td class="tg-dzk6">3.07</td>
  </tr>
  <tr>
    <td class="tg-baqh">Linear Regression</td>
    <td class="tg-baqh">3.78</td>
  </tr>
  <tr>
    <td class="tg-dzk6">KNN</td>
    <td class="tg-dzk6">4.17</td>
  </tr>
  <tr>
    <td class="tg-baqh">Random Forest</td>
    <td class="tg-baqh">3.09</td>
  </tr>
  <tr>
    <td class="tg-dzk6">Gradient Boosting</td>
    <td class="tg-dzk6">3.63</td>
  </tr>
  <tr>
    <td class="tg-baqh">Mean of all regressors</td>
    <td class="tg-baqh">3.37</td>
  </tr>
</tbody>
</table></div></center>

In [ ]:
# make subplot with predictions plots
fig = make_subplots(
                    rows=2, 
                    cols=3, 
                    horizontal_spacing = 0.005, 
                    vertical_spacing = 0.01, 
                    shared_yaxes=True, 
                    shared_xaxes=True
                    )

# add plots to the graph
for t in fig_tree.data:
    fig.append_trace(t, row=1, col=1)

for t in fig_lm.data:
    fig.append_trace(t, row=1, col=2)

for t in fig_knn.data:
    fig.append_trace(t, row=1, col=3)

for t in fig_rcf.data:
    fig.append_trace(t, row=2, col=1)

for t in fig_gb.data:
    fig.append_trace(t, row=2, col=2)

for t in fig_all.data:
    fig.append_trace(t, row=2, col=3)

# add axes titles
fig.update_yaxes(title_text='Sales', row=1, col=1,)
fig.update_yaxes(title_text='Sales', row=2, col=1)
fig.update_xaxes(title_text='Date', row=2, col=1)
fig.update_xaxes(title_text='Date', row=2, col=2)
fig.update_xaxes(title_text='Date', row=2, col=3)

# modify layout of the graph 
fig.update_layout(
                  font_family="'Nunito', sans-serif",
                  title={'text': "Predictions of different ML models", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  margin=dict(l=40, r=10, t=60, b=80),
                  legend=dict(
                              orientation='h',
                              yanchor='middle',
                              y=-0.2,
                              xanchor='center',
                              x=0.5,
                              )
                  )
fig.show()

<br> 
<h2 id="Prediction">Prediction</h2>  

--- 

<div align="justify">For predictions of df_test data Decision Tree model is used. To visualise predicted values, a graph was created that shows the mean daily sales in each city. This graph looks similar to what was observed earlier in exploratory data analysis: the same difference of rates of sales between cities; and the same tendency throughout the week.
Below, there is the head of the dataframe with number of predicted sales.</div>

In [ ]:
# Tree model for actual test data
tree_clf = DecisionTreeRegressor(max_depth=12).fit(df_train, sales)
forecast = tree_clf.predict(df_test)


forecast1 = pd.DataFrame(forecast, columns = ['prediction'])
id = pd.DataFrame(df_test.id, columns = ['id'])

forecast1.reset_index(drop=True, inplace=True)
id.reset_index(drop=True, inplace=True)

result = pd.concat([id, forecast1], axis=1)

In [ ]:
cities = list(group.city_name.unique())

prediction = pd.merge(result, test[['id', 'date', 'city_name']], how="left", on=['id'])

prediction = prediction.replace(            
    ('Москва', 'Санкт-Петербург', 'Краснодар', 'Самара','Нижний Новгород', 'Ростов-на-Дону', 'Волгоград', 'Воронеж', 'Казань', 'Екатеринбург'),
    ("Moscow", "St.Petersburg", "Krasnodar", "Samara", "Nizhny.Novgorod", "Rostov-on-Don", "Volgograd", "Voronezh", "Kazan", "Yekaterinburg")  )


prediction.date = pd.to_datetime(prediction.date)                              # convert date column to date format    


In [ ]:
prediction = prediction.groupby(['date', 'city_name'], as_index=False).sum()
fig = px.line(
              prediction, 
              x="date", 
              y="prediction", 
              color='city_name'
              )
fig.update_layout(
                  font_family="'Nunito', sans-serif",
                  title={'text': "Sum of predicted sales for the week", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  xaxis_title='Date',
                  yaxis_title='Sales',
                  legend=dict(y=0.5, title="City")
                  )
fig.show() 

In [ ]:
# # the code is represented in the table below
# result.head()

<center>
<p>The first 5 rows of the dataframe with predicted sales</p>
<style type="text/css">
.tg  {border-collapse:collapse;border-color:#ccc;border-spacing:0;}
.tg td{background-color:#fff;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#f0f0f0;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-dzk6{background-color:#f9f9f9;text-align:center;vertical-align:top}
@media screen and (max-width: 767px) {.tg {width: auto !important;}.tg col {width: auto !important;}.tg-wrap {overflow-x: auto;-webkit-overflow-scrolling: touch;}}</style>
<div class="tg-wrap"><table class="tg">
<thead>
  <tr>
    <th class="tg-amwm">id</th>
    <th class="tg-amwm">prediction</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-dzk6">666677</td>
    <td class="tg-dzk6">17</td>
  </tr>
  <tr>
    <td class="tg-baqh">666678</td>
    <td class="tg-baqh">28</td>
  </tr>
  <tr>
    <td class="tg-dzk6">666679</td>
    <td class="tg-dzk6">26</td>
  </tr>
  <tr>
    <td class="tg-baqh">666680</td>
    <td class="tg-baqh">22</td>
  </tr>
  <tr>
    <td class="tg-dzk6">666681</td>
    <td class="tg-dzk6">25</td>
  </tr>
</tbody>
</table></div></center>

In [ ]:
# # download the dataframe with predicted sales
# result.to_csv("prediction.csv", index=False)
# files.download("prediction.csv")

<br> 
<h2 id="Results">Results</h2>  

--- 

<div align="justify">In this project, sales predictions were made for the stores across 10 russian cities for the 3rd weeek of February.  

<ul>
  <li>Exploratory analysis revealed patterns in data, which can be easily interpreted with russian population and salary distribution statistics.</li>
  <li>Correlation between data and sales rates was improved by use one-hot encoding for object data, like city names.</li>
  <li>Correlation was further improved by use of lag features of sales. The highest correlation was observed for mean number of sales for each day of the week for products in each store (day_product_mean) and mean of three lag days (lag_days_mean): 0.90 and 0.88 respectively.</li>
  <li>6 ML models were tested and evaluated using MAE score. The Decision Tree model had the lowest MAE score 3.07 and was chosen for predictions.</li>
</ul>  </div>
  

<br><br><br>
<center><a href="https://konstantinburkin.github.io">Main page</a></center><br><br>